In [255]:
import numpy as np
import sys
sys.path.insert(0,"../")
import data_generator as gen
import copy

new = np.newaxis

debug = False
dprint = print if debug else lambda *args, **kwargs : None


In [479]:
class Tensor():
    
    def __init__(self,elem=None, shape=None, axes_names=None, sigma=1e-3):
        
        # Numeric initialization
        if (elem is None) and (shape is not None):
            #self.elem = np.random.normal(scale=sigma, size=shape)
            self.elem = np.random.random(size=shape)
            fact = np.sqrt(self.elem.flatten().shape[0])
            #print("fact")
            #print(fact)
            self.elem /= fact/np.sqrt(2)
            #self.elem /= np.abs(self.elem).sum()
            #self.elem *= 6
        elif elem is not None:
            self.elem = elem
        else:
            raise Exception('You have to provide either the elements of the tensor or its shape')
            
        # Relevant attributes initialization
        self.shape = self.elem.shape
        self.rank = len(self.shape)
        self.aggregations = {}

        if axes_names is not None:
            try:
                if len(axes_names) == self.rank:
                    self.history_axes_names = [np.array(axes_names)]
                    self.axes_names = np.array(axes_names)
                else:
                    raise ValueError("sddsd")
            except TypeError:
                print("=== Warning ===\nThe object that describes the indexes names have at least to support the built-in len function."\
                          +"\naxes_names attribute has not been inizialized.")
                self.axes_names = None
            except ValueError:
                print("=== Warning ===\nThe number of names should match the rank of the tensor."\
                          +"\naxes_names attribute has not been inizialized.")
                self.axes_names = None
        else:
            self.axes_names = None

        return


    def transpose(self, permutation):
        indexes = self.ax_to_index(permutation)
        self.elem = np.transpose(self.elem, indexes)
        self.update_members(permutation)
        return

    def ax_to_index(self, axes):
        # handle single and multiple indeces differently
        if type(axes) == str:
            return np.where(self.axes_names == axes)[0][0]
        else:
            return_axes = []
            for ax in axes:
                return_axes.append(np.where(self.axes_names == ax)[0][0])
            return return_axes

    
    def aggregate(self, axes_names=None, new_ax_name=None):
        """ 
        Utilization: ...
        """
        # Sanity checks
        if (axes_names is None) and (new_ax_name is not None):
            axes_names = self.axes_names
        elif new_ax_name is None:
            raise ValueError("You have to provide the name of the new axes")
            
        if self.axes_names is None:
            raise ValueError("This function can be called only if the axes names are defined")
            
        for name in axes_names:
            assert name in self.axes_names, "The " + name + " axes wasn't found in the tensor"
            
        dprint("Aggregating...")


        # Convert the axes names to their index positions
        indexes = self.ax_to_index(axes_names)
        
        # Store original shape of the aggregated indexes
        axes_sizes = np.array(self.shape)[indexes]
        self.aggregations[new_ax_name] = dict(zip(axes_names, axes_sizes))
        
        # Gather the non contracted indexes
        all_indexes = set(range(len(self.elem.shape)))
        other_indexes = list(all_indexes.difference(set(indexes)))
        other_indexes.sort()

        dprint("axes_numerical+other_axes: ", indexes+other_indexes)

        # Perform actual reshaping
        self.elem = np.transpose(self.elem, indexes+other_indexes)        
        other_sizes = np.array(self.shape)[other_indexes].tolist()
        self.elem = self.elem.reshape([-1]+other_sizes)
        
        # Update class members
        self.update_members(np.concatenate([[new_ax_name], self.axes_names[other_indexes]]))
        
        return
        

    def disaggregate(self, ax):
        """
        
        """
        
        assert ax in self.axes_names, "The " + ax + " ax wasn't found in the tensor."
        assert ax in self.aggregations.keys(), "The " + ax + " do not represent an aggregated ax."
        
        original_dict = self.aggregations[ax]
        original_names = list(original_dict.keys())
        original_shape = list(original_dict.values())
        
        index = self.ax_to_index(ax)
        
        # transpose to have the aggregated index at the beginning
        permutation = [index] + np.arange(index).tolist() + np.arange(index+1, self.rank).tolist()
        self.elem = np.transpose(self.elem, permutation)
        self.update_members(self.axes_names[permutation])
        
        # Disaggregate axis by reshaping the tensor
        self.elem = self.elem.reshape(original_shape + list(self.shape[1:]))
        self.update_members(np.concatenate([original_names, self.axes_names[1:]]))
        
        # Remove aggregated index from the memory
        self.aggregations.pop(ax)
        
        return

    def update_members(self, axes_names):
        self.axes_names = np.array(axes_names)
        self.shape = self.elem.shape
        self.rank = len(self.shape)
        return
    
    def check_names(self):
        print("="*10+"axes_names type"+"="*10)
        print(type(self.axes_names))
        
    def __str__(self):
        print("="*10+" Tensor description "+"="*10)
        print("Tensor shape: ", self.shape)
        print("Tensor rank: ", self.rank)
        print("Axes names: ", self.axes_names)
        return ""

In [480]:
t = Tensor(shape=[2,3,2], axes_names=['i','j','k'])
print("Original tensor")
print('t.axes_names: ', t.axes_names)
print('t.shape: ', t.shape)
print('t.elem: ', t.elem)
t.aggregate(axes_names=['j','i'], new_ax_name='l')
print('\nAfter first aggregation:')
print('t.axes_names: ', t.axes_names)
print('t.shape: ', t.shape)
print('t.elem: ', t.elem)
t.aggregate(axes_names=['l','k'])
print('\nAfter second aggregation:')
print('t.axes_names: ', t.axes_names)
print('t.shape: ', t.shape)
print('t.elem: ', t.elem)
t.disaggregate()
print('\nAfter disaggregation:')
print('t.axes_names: ', t.axes_names)
print('t.shape: ', t.shape)
print('t.elem: ', t.elem)

Original tensor
t.axes_names:  ['i' 'j' 'k']
t.shape:  (2, 3, 2)
t.elem:  [[[0.28594366 0.0778324 ]
  [0.30659933 0.0847331 ]
  [0.13723071 0.31311426]]

 [[0.05888709 0.3531357 ]
  [0.18669282 0.28609468]
  [0.30465577 0.35132249]]]

After first aggregation:
t.axes_names:  ['l' 'k']
t.shape:  (6, 2)
t.elem:  [[0.28594366 0.0778324 ]
 [0.05888709 0.3531357 ]
 [0.30659933 0.0847331 ]
 [0.18669282 0.28609468]
 [0.13723071 0.31311426]
 [0.30465577 0.35132249]]


ValueError: You have to provide the name of the new axes

In [481]:
def _contract_(T1, T2, contracted_axis1, contracted_axis2, common_axis1=[], common_axis2=[]):
    
    # Sanity checks
    assert len(common_axis1) == len(common_axis2), "number of common axes is different"
    
    if type(contracted_axis1) != list:
        assert T1.shape[contracted_axis1] == T2.shape[contracted_axis2], "dimensions of contracted axes do not match"
        contracted_axis1 = [contracted_axis1]
        contracted_axis2 = [contracted_axis2]
    
    for i in range(len(common_axis1)):
        assert T1.shape[common_axis1[i]] == T2.shape[common_axis2[i]], "dimensions of common axes do not match"
        
    original_shape1 = np.array(T1.shape)
    original_shape2 = np.array(T2.shape)
        
    def perm(contracted_axis, original_shape, common_axis):
        
        # astype is for handle the case in the first array is empty, in which the function cannot infere the type
        last_axis = np.concatenate((common_axis, contracted_axis)).astype("int64")         

        remaining_axis = np.delete(np.arange(len(original_shape)), last_axis)
        permutation = np.concatenate((remaining_axis, last_axis))
        return permutation

    permutation1 = perm(contracted_axis1, original_shape1, common_axis1)
    permutation2 = perm(contracted_axis2, original_shape2, common_axis2)

    shape1 = original_shape1[permutation1]
    shape2 = original_shape2[permutation2]

    #print(shape2)
    
    #inverse1 = get_inverse_permutation(permutation1)
    # param for match the rank of the two shapes
    unique1 = len(shape1)-len(common_axis1)-len(contracted_axis1)
    unique2 = len(shape2)-len(common_axis1)-len(contracted_axis1)

    new_shape1 = np.concatenate((shape1[:unique1],[1 for i in range(unique2)],shape1[unique1:])).astype("int64")
    new_shape2 = np.concatenate(([1 for i in range(unique1)],shape2)).astype("int64")

    #print(new_shape1)

#    print(new_shape1)
    #print(new_shape2)
    T1.elem = np.transpose(T1.elem, permutation1)
    T2.elem = np.transpose(T2.elem, permutation2)
    
    if T1.axes_names is not None:
        #print(T1.axes_names)
        T1.axes_names = T1.axes_names[permutation1]
        T2.axes_names = T2.axes_names[permutation2]
        T3_axes_names = np.concatenate([T1.axes_names[:unique1], T2.axes_names[:T2.rank-len(contracted_axis2)]])
        #print(T3_axes_names)
    else: 
        T3_axes_names = None

    T3 = (T1.elem.reshape(new_shape1)*T2.elem.reshape(new_shape2))
    if len(contracted_axis1) > 0:
        T3 = T3.sum(axis=-1)
        
    T3 = Tensor(elem=T3, axes_names=T3_axes_names)
    return T3

def contract(T1, T2, contracted_axis1=[], contracted_axis2=[], common_axis1=[], common_axis2=[], contracted=None, common=None):
        
    if contracted is not None:
        contracted_axis1 = contracted
        contracted_axis2 = contracted
 
    if common is not None:
        common_axis1 = common
        common_axis2 = common
        
    if type(common_axis1) == int:
        common_axis1 = [common_axis1]
    if type(common_axis2) == int:
        common_axis2 = [common_axis2]

    if type(contracted_axis1) == str:
        contracted_axis1 = np.where(T1.axes_names == contracted_axis1)[0][0]

    if type(contracted_axis2) == str:
        contracted_axis2 = np.where(T2.axes_names == contracted_axis2)[0][0]
    
    temp = []
    for key in common_axis1:
        if type(key) == str:
            temp.append(np.where(T1.axes_names == key)[0][0])
        else:
            temp.append(key)
    common_axis1 = temp

    temp = []
    for key in common_axis2:
        if type(key) == str:
            temp.append(np.where(T2.axes_names == key)[0][0])
        else:
            temp.append(key)
    common_axis2 = temp
    
    return _contract_(T1, T2, contracted_axis1, contracted_axis2, common_axis1, common_axis2)

In [482]:
x = np.random.random([1,2,3,4])
y = np.random.random([3,4,5,6])
T1 = Tensor(x, axes_names=['i','j','k','l'])
T2 = Tensor(y, axes_names=['k','l','n','m'])
T3 = contract(T1,T2, contracted='k', common='l')
print(T1)

========== Tensor description ==========
Tensor shape:  (1, 2, 3, 4)
Tensor rank:  4
Axes names:  ['i' 'j' 'l' 'k']



In [483]:
print(T1)

========== Tensor description ==========
Tensor shape:  (1, 2, 3, 4)
Tensor rank:  4
Axes names:  ['i' 'j' 'l' 'k']



In [484]:
print(T1)

T1.aggregate2(["i","j"], new_ax_name="bla")
print(T1)
print(T1.aggregations)

T1.aggregate2(["bla","k"], new_ax_name="blabla")
print(T1)
print(T1.aggregations)

T1.disaggregate2('blabla')
print(T1)
print(T1.aggregations)

T1.disaggregate2('bla')
print(T1)
print(T1.aggregations)

ll = T1

T1.transpose(['i','k','l','j'])

print(T1)
print(T1.aggregations)

T1.transpose(['i','j','l','k'])
print(np.sum(ll.elem-T1.elem))

========== Tensor description ==========
Tensor shape:  (1, 2, 3, 4)
Tensor rank:  4
Axes names:  ['i' 'j' 'l' 'k']



AttributeError: 'Tensor' object has no attribute 'aggregate2'

In [485]:
class Network():
    
    def __init__(self, N, M, D=2, L=10, sigma=1e-2):
        
        self.N = N
        self.D = D
        self.L = L
        
        self.As = []
        
        self.As.append(Tensor(shape=[L,M,D], axes_names=['l','right','d0'], sigma=sigma))
        for i in range(1,N-1):
            self.As.append(Tensor(shape=[M,M,D], axes_names=['left','right','d'+str(i)], sigma=sigma))
        self.As.append(Tensor(shape=[M,D], axes_names=['left','d'+str(N-1)], sigma=sigma))
        
        self.l_pos = 0

        
    def forward(self, X, train=False):
        
        assert self.N == X.shape[1], "The 1 dimension of the input data must be the flattened number of pixels"

        # X should be batch_size x 784 x 2
        TX = []
        for i in range(self.N):
            TX.append(Tensor(elem=X[:,i,:], axes_names=['b','d'+str(i)]))
                      
        # This must be futher investigate, three ways:
        #     * numpy vectorize
        #     * list comprehension
        #     * multithread
                      
        #A_TX = np.vectorize(contract)(TX, A, contracted='d'+str(i))
        A_TX = [contract(self.As[i], TX[i], contracted='d'+str(i)) for i in range(self.N)]
        print('check')
        cum_contraction = []
        cum_contraction.append(A_TX[-1])
        for j in range(self.N-1):
            tmp_cum = copy.deepcopy(cum_contraction[-1])
            #print("tmp_cum")
            #print(tmp_cum)
            #t = Tensor(elem=tmp_cum.elem, axes_names=tmp_cum.axes_names)
            tmp_cum = contract(A_TX[-(j+2)], tmp_cum, 'right', 'left', common='b')
            
            cum_contraction.append(tmp_cum)
            #print("tmp_cum")
            #print(tmp_cum)
            #print("cum_contraction[-1]")
            #print(cum_contraction[-1])
            #if j > 1:
            #    print("cum_contraction[-2]")
            #    print(cum_contraction[-2])
        #print(cum_contraction[3])
        #print("cum_contraction[-1]")
        #print(cum_contraction[-1])

        if train:
            self.cum_contraction = cum_contraction
            self.TX = TX


        return cum_contraction[-1]

    def sweep_step(self, f, y, lr):
        l = self.l_pos
        
        B = contract(self.As[l], self.As[l+1], "right", "left")

        f.elem = y-f.elem

        phi = contract(self.TX[l], self.TX[l+1], common="b")

        if l>0:
            pass
        

        if l<(self.N-2):
            phi = contract(phi, self.cum_contraction[-(l+2)], common = "b")
        
        deltaB = contract(f, phi, contracted="b")

        B.transpose(['l', 'd0', 'd1', 'right'])
        B.elem = B.elem + lr*deltaB.elem
        
        B.aggregate(axes_names=['d1','right','l'], new_ax_name='j')

        B.transpose(['d0','j'])
        print(B)
        #np.linalg.svd(B.elem)
        
        #B.disaggregate()
        
        return B
    
    def sweep(self, X, y, lr):
        #g = self.forward(np.ones(X.shape), train = True)
        #print("="*100)
        #for A in self.cum_contraction:
        #    print(A.elem.sum())
        #print("="*100)
        #print(g.elem)
        f = self.forward(X, train = True)
        one_hot_y = np.zeros((y.size, self.L))
        one_hot_y[np.arange(y.size),y] = 1
        y = one_hot_y.T
        B = self.sweep_step(f, y, lr)
        return B
    
    def train():
        return

In [492]:
(data, label) = gen.create_dataset(5)
x = data.reshape(5,25)

def psi(x):
    x = np.array((np.sin(np.pi*x/2),np.cos(np.pi*x/2)))
    return np.transpose(x, [1,2,0])

x = psi(x)

In [493]:


# Batch normalization
#x = (x-x.mean(axis=0))/x.std(axis=0)

net = Network(N=25, M=100, L=2)#, sigma=0.23)

#out = net.forward(x)

B = net.sweep(x, label, lr=0.1)


check
========== Tensor description ==========
Tensor shape:  (2, 400)
Tensor rank:  2
Axes names:  ['d0' 'j']



In [494]:
5/(100)**(25-1) # b/(m)**(N-1)

5e-48

In [495]:
print(B.history_axes_names)
print(B.permutation_history)
print(B.transposed_shape)

[array(['l', 'd0', 'right', 'd1'], dtype='<U5')]


AttributeError: 'Tensor' object has no attribute 'permutation_history'

In [496]:
def tensor_svd(T):
    
    if type(T) != Tensor:
        raise TypeError("This function only support object from the class Tensor")

    if len(T.shape) != 2:
        raise ValueError("This function only support a 2D tensors")

    print(T.aggregations)
    SVD = np.linalg.svd(T.elem)
    U = SVD[0]
    S = SVD[1]
    Vh = SVD[2]

    print(U.shape)
    print(Vh.shape)
    # cut
    Vh = Vh[:2,:]
    
    print(Vh.shape)
    SVh = Vh*S[:,new]
    print(SVh.shape)
    TSVh = Tensor(elem=SVh, axes_names=['left','j'])
    TSVh.aggregations = T.aggregations
    #TSVh.transpose(['j','d0'])
    TSVh.disaggregate('j')
    
    print(TSVh)
    
    return 

tensor_svd(B)


{'j': {'d1': 2, 'right': 100, 'l': 2}}
(2, 2)
(400, 400)
(2, 400)
(2, 400)
permutation
[1, 0]
========== Tensor description ==========
Tensor shape:  (2, 100, 2, 2)
Tensor rank:  4
Axes names:  ['d1' 'right' 'l' 'left']



In [338]:
np.cumsum(S)/S.sum()


NameError: name 'S' is not defined

In [83]:
Vh*S[:,np.newaxis]

NameError: name 'Vh' is not defined

In [65]:
l = ['i','j','k']
m = [0,2,1]
p = {}
for i in range(len(l)):
    p[l[i]] = m[i]
    

In [69]:
kh = dict(zip(l, m))


In [110]:
np.prod(kh.values())

dict_values([0, 2, 1])

In [111]:
np.prod([2,3])

6

In [178]:
l = [1,2,3,4]
l[[0,1]]

TypeError: list indices must be integers or slices, not list

In [322]:
x = {'l':2}
if 'm' in x.keys():
    print("ok")

In [324]:
x.pop('l')

2

In [325]:
x

{}

In [ ]:
BACKUP

            for key in kwargs.keys():
                if type(kwargs[key][0]) == str:
                    axes_numerical = []
                    for index in kwargs[key]:
                        if self.axes_names is not None:
                            idx = np.where(self.axes_names == index)[0][0]
                            axes_numerical.append(idx)
                        else:
                            raise ValueError("=== Error ===\nThe function was called in the index mode but the tensor wasn't initializated with index names. ")
                    
            all_axes = set(range(self.rank))
            #!!!!!!!!!!!!!!!! check without list AND sort
            other_axes = list(all_axes.difference(set(axes_numerical)))
            other_axes.sort()
            
            final_list = []
            for indexes in list_indexes:
                final_list += indexes
            final_list += other_axes
                    
            self.elem = np.transpose(self.elem, axes_numerical+other_axes)
                print(axes_numerical)
                else:
                    print()
            print()


In [79]:
    """def aggregate(self, axes_numerical=None, axes_names=None, new_ax_name=None):
            """ 
            #Utilization: ...
            """
            dprint = print if debug else lambda *args, **kwargs : None
            dprint("Aggregating...")

            if (axes_names is None) and (axes_numerical is None):
                self.elem = self.elem.reshape(-1)
                self.shape = self.elem.shape
                self.history_axes_names.append(np.array([new_ax_name]))
                self.axes_names = np.array([new_ax_name])
                return

            if axes_names is not None:
                if axes_numerical is None:
                    axes_numerical = []
                    for name in axes_names:
                        if type(name) == str:
                            if self.axes_names is not None:
                                dprint('name: ', name)
                                dprint('self.axes_names: ', self.axes_names)
                                dprint(np.where(self.axes_names == name))
                                idx = np.where(self.axes_names == name)[0][0]
                                axes_numerical.append(idx)
                            else:
                                raise ValueError("=== Error ===\nThe function was called in the index mode but the tensor wasn't initializated with index names. ")

                    #self.axes_names = np.array([new_ax_name] + list(set(self.axes_names).difference(set(axes_names))))
                    #self.history_axes_names.append(self.axes_names) 
                else:
                    raise Exeption("=== Error ===\nProvide just one input between axes_numerical and axes_names.")


            if (new_ax_name is not None) and (self.axes_names is not None):

                # test this case with new_ax_names and axes_numerical and not axes_names
                if axes_names is None:
                    axes_names = self.axes_names[axes_numerical]

                self.axes_names = np.array([new_ax_name] + list(set(self.axes_names).difference(set(axes_names))))
                self.history_axes_names.append(self.axes_names) 

            all_axes = set(range(len(self.elem.shape)))
            #!!!!!!!!!!!!!!!! check without list AND sort
            other_axes = list(all_axes.difference(set(axes_numerical)))
            other_axes.sort()

            dprint("axes_numerical+other_axes: ", axes_numerical+other_axes)
            self.elem = np.transpose(self.elem, axes_numerical+other_axes)
            self.transposed_shape.append(self.elem.shape)

            self.permutation_history.append(axes_numerical+other_axes)
            self.elem = self.elem.reshape([-1]+list(self.transposed_shape[-1][len(axes_numerical):]))
            self.shape = self.elem.shape

            return

        def disaggregate(self, index):
            """

            """

            try:
                if len(self.permutation_history) > 0:
                    while len(self.permutation_history) > 0:
                        # get and invert last permutation of shape
                        last_permutation = list(self.permutation_history[-1])
                        last_rank = len(last_permutation)
                        permutation_matrix = np.zeros([last_rank,last_rank])
                        permutation_matrix[range(last_rank),last_permutation] = 1
                        # disaggregate using last transposed shape
                        self.elem = self.elem.reshape(self.transposed_shape[-1])
                        # IDEA: original_shape = np.dot(permutation_matrix.T, self.transposed_shape)
                        inverse_permutation = np.where(permutation_matrix.T == 1)[1]
                        self.elem = np.transpose(self.elem, inverse_permutation)
                        # remove history of last aggregation
                        self.permutation_history.pop()
                        self.transposed_shape.pop()

                        if len(self.history_axes_names) > 1:
                            self.history_axes_names.pop()
                            self.axes_names = self.history_axes_names[-1]

                        self.shape = self.elem.shape
                    return
            except:
                print("Cannot disaggregate tensor if it was not aggregated in the first place.")
                return"""
    

ciao


In [ ]:
def f(x,y):
    return

f(*par)